In [1]:
# Комментарий: Импортируются все необходимые библиотеки для загрузки данных, 
# предобработки, обучения моделей и оценки их качества.
import pandas as pd
import numpy as np
import optuna  # для оптимизации гиперпараметров XGBoost
from sklearn.model_selection import StratifiedKFold, cross_val_score  # для кросс-валидации
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # для предобработки данных
from sklearn.compose import ColumnTransformer  # для комбинирования разных преобразований
from sklearn.pipeline import Pipeline  # для создания пайплайна обработки данных
from sklearn.metrics import accuracy_score  # для оценки точности
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # модели машинного обучения
from sklearn.linear_model import LogisticRegression  # финальная модель в Stacking
from sklearn.impute import SimpleImputer  # для заполнения пропусков
from lightgbm import LGBMClassifier  # модель LightGBM
from xgboost import XGBClassifier  # модель XGBoost

# Загрузка данных
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("\nTarget distribution:")
print(train['Transported'].value_counts(normalize=True))


Train shape: (8693, 14)
Test shape: (4277, 13)

Target distribution:
Transported
True     0.503624
False    0.496376
Name: proportion, dtype: float64


In [2]:
# Анализ пропусков
print("Пропуски в train:")
print(train.isnull().sum())
# Распределение числовых признаков
train.select_dtypes(include=[np.number]).describe()
# Взаимосвязь Age и Transported
train['AgeBin'] = pd.cut(train['Age'], bins=[0,12,18,35,60,100])
age_transported = train.groupby('AgeBin', observed=True)['Transported'].mean()  # Явно указываем observed=True
print(age_transported)


Пропуски в train:
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64
AgeBin
(0, 12]      0.668790
(12, 18]     0.537299
(18, 35]     0.467602
(35, 60]     0.487135
(60, 100]    0.472727
Name: Transported, dtype: float64


#### Функция preprocess выполняет комплексную предобработку данных:
извлекает информацию из столбца Cabin;

создаёт новые признаки на основе Name и PassengerId;

суммирует траты по категориям и создаёт бинарный признак HasSpend;

применяет логарифмирование к тратам для устойчивости к нулевым значениям;

создаёт бинарные признаки для пропусков;

категоризирует возраст;

создаёт признак взаимодействия между CryoSleep и тратами.

In [3]:
spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
def preprocess(df, is_train=True):
    df = df.copy()
    
    # 1. Извлечение из Cabin
    df[['Deck', 'CabinNum', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
    df['CabinNum'] = pd.to_numeric(df['CabinNum'], errors='coerce')
    
    # 2. Извлечение из Name
    df['LastName'] = df['Name'].str.split(' ').str[-1]
    
    # 3. Групповые признаки
    df['GroupId'] = df['PassengerId'].str.split('_').str[0]
    df['GroupSize'] = df.groupby('GroupId')['PassengerId'].transform('count')
    
    # 4. Суммарные траты
    spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df['TotalSpend'] = df[spend_cols].sum(axis=1, skipna=True)
    df['HasSpend'] = (df['TotalSpend'] > 0).astype(int)
    
    # 5. Логарифмы трат (устойчиво к 0)
    for col in spend_cols:
        df[f'Log_{col}'] = np.log1p(df[col].fillna(0))
    
    # 6. Бинарные признаки пропусков
    for col in spend_cols + ['CryoSleep', 'VIP']:
        df[f'{col}_isnull'] = df[col].isna().astype(int)
    bool_to_str_cols = ['CryoSleep', 'VIP']
    for col in bool_to_str_cols:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    # 7. Возрастные категории
    df['AgeGroup'] = pd.cut(df['Age'], [0,12,18,35,60,120], 
                              labels=['child','teen','young','adult','senior'])
    
    # 8. Взаимодействие: Cryo + нулевые траты
    df['Cryo_ZeroSpend'] = ((df['CryoSleep'] == True) | 
                             (df['TotalSpend'].fillna(0) == 0)).astype(int)
    
    return df
# Применяем функцию предобработки к тренировочному и тестовому датасетам.
train_proc = preprocess(train)
test_proc = preprocess(test, is_train=False)


In [4]:
# Разделяем признаки на числовые (num_cols), категориальные (cat_cols) 
# и бинарные (bool_cols). 
# Формируем итоговый список признаков all_features и разделяем данные на 
# признаки (X, X_test) и целевой признак (y).
num_cols = ['Age', 'TotalSpend', 'CabinNum'] + [f'Log_{c}' for c in spend_cols]
cat_cols = ['HomePlanet', 'Destination', 'Deck', 'CabinSide', 'AgeGroup', 'CryoSleep', 'VIP']
bool_cols = [c for c in train_proc.columns if c.endswith('_isnull')] + ['HasSpend', 'Cryo_ZeroSpend']

all_features = num_cols + cat_cols + bool_cols

X = train_proc[all_features]
y = train['Transported'].astype(int)
X_test = test_proc[all_features]

# Создаём пайплайн предобработки данных:

# для числовых признаков: заполнение пропусков медианой и стандартизация;

# для категориальных признаков: заполнение пропусков и one-hot кодирование;

# бинарные признаки оставляем без изменений.
# Препроцессор
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols),
    ('bool', 'passthrough', bool_cols)
])


In [5]:
# LightGBM — градиентный бустинг на деревьях, хорошо работает с табличными данными. 
# Параметры подобраны для баланса между скоростью обучения и качеством модели.
models = {}

# Модель 1: LightGBM
models['lgb'] = Pipeline([
    ('pre', preprocessor),
    ('clf', LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.85,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        verbose=-1
    ))
])

In [6]:
# Модель 2: XGBoost с Optuna
# XGBoost — ещё один градиентный бустинг, часто даёт результаты, 
# сопоставимые с LightGBM. Гиперпараметры оптимизируются с помощью Optuna — 
# библиотеки для автоматической настройки гиперпараметров. 
# Это позволяет найти оптимальные параметры для данной задачи.
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.95),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 1.0),
    }
    model = Pipeline([
        ('pre', preprocessor), 
        ('clf', XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric='logloss'))
    ])
    cv_scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv_scores, scoring='accuracy')
    return np.mean(scores)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)
best_xgb_params = study_xgb.best_params

models['xgb'] = Pipeline([
    ('pre', preprocessor),
    ('clf', XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric='logloss'))
])


[I 2025-11-08 08:46:52,860] A new study created in memory with name: no-name-4bd54e17-8dc0-41ee-a0c0-e05073ede443
[I 2025-11-08 08:47:03,199] Trial 0 finished with value: 0.8048997777249864 and parameters: {'n_estimators': 975, 'max_depth': 6, 'learning_rate': 0.04635720523200844, 'subsample': 0.7491971017275098, 'colsample_bytree': 0.821941684939788, 'reg_lambda': 0.6810845309369341}. Best is trial 0 with value: 0.8048997777249864.
[I 2025-11-08 08:47:13,754] Trial 1 finished with value: 0.804440603471037 and parameters: {'n_estimators': 841, 'max_depth': 7, 'learning_rate': 0.06015355479274368, 'subsample': 0.8519883439204333, 'colsample_bytree': 0.8907966345260211, 'reg_lambda': 0.5246821177113166}. Best is trial 0 with value: 0.8048997777249864.
[I 2025-11-08 08:47:33,198] Trial 2 finished with value: 0.804439809395371 and parameters: {'n_estimators': 1461, 'max_depth': 7, 'learning_rate': 0.024675974040332727, 'subsample': 0.7017538251889606, 'colsample_bytree': 0.8419388918317947

In [7]:
# Модель 3: Random Forest
# Random Forest — ансамбль решающих деревьев, 
# менее чувствителен к выбросам и переобучению по сравнению с бустингом. 
# Используется как базовая модель для сравнения и в ансамбле
models['rf'] = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(
        n_estimators=500,
        max_depth=12,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ))
])

In [8]:
# Модель 4: Stacking
# Stacking — метод ансамблирования, где предсказания базовых моделей 
# (lgb, xgb, rf) используются как признаки для финальной модели (LogisticRegression). 
# Это часто даёт прирост качества по сравнению с отдельными моделями.
estimators = [
    ('lgb', models['lgb']),
    ('xgb', models['xgb']),
    ('rf', models['rf'])
]

models['stack'] = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=42),
    cv=5,
    n_jobs=-1
)

In [9]:
# 7. Обучение и валидация моделей
# Для каждой модели выполняется кросс-валидация (5 фолдов) 
# для оценки качества и полное обучение на всём тренировочном наборе.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models.items():
    print(f"\n--- Обучение модели: {name} ---")
    
    # Кросс‑валидация
    try:
        cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
        print(f"CV Accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
        results[name] = np.mean(cv_scores)
    except Exception as e:
        print(f"Ошибка при кросс-валидации для {name}: {e}")
        continue

    # Полное обучение на всём тренировочном наборе
    model.fit(X, y)


--- Обучение модели: lgb ---
CV Accuracy: 0.8074 ± 0.0071

--- Обучение модели: xgb ---
CV Accuracy: 0.8133 ± 0.0072

--- Обучение модели: rf ---
CV Accuracy: 0.8012 ± 0.0056

--- Обучение модели: stack ---
CV Accuracy: 0.8060 ± 0.0092


/usr/local/lib/python3.11/dist-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)


In [10]:
# Вывод результатов CV
print("\n--- Итоги кросс‑валидации ---")
for name, score in results.items():
    print(f"{name}: {score:.4f}")

# 8. Предсказания на тестовом наборе и сохранение submission
submission_dir = './submissions'
import os
if not os.path.exists(submission_dir):
    os.makedirs(submission_dir)


# Создаём шаблон для submission (используем PassengerId из test)
submission_template = test[['PassengerId']].copy()


for name, model in models.items():
    print(f"Генерируем предсказания для {name}...")
    
    preds = model.predict(X_test)
    submission = submission_template.copy()
    submission['Transported'] = preds.astype(bool)
    
    
    # Сохраняем в CSV
    submission.to_csv(f"{submission_dir}/submission_{name}.csv", index=False)
    print(f"Сохранено: {submission_dir}/submission_{name}.csv")


# 9. Ансамблевое предсказание (среднее по вероятностям)
print("\n--- Генерируем ансамблевый submission ---")


ensemble_preds = np.zeros(len(X_test))


for name, model in models.items():
    if hasattr(model, 'predict_proba'):
        try:
            proba = model.predict_proba(X_test)[:, 1]  # вероятность класса 1
        except:
            print(f"Не удалось получить вероятности для {name}, используем predict")
            proba = model.predict(X_test).astype(float)
    else:
        proba = model.predict(X_test).astype(float)
    ensemble_preds += proba

# Среднее по моделям
ensemble_preds /= len(models)


# Бинаризация: порог 0.5
ensemble_binary = (ensemble_preds >= 0.5).astype(int)


submission_ensemble = submission_template.copy()
submission_ensemble['Transported'] = ensemble_binary.astype(bool)
submission_ensemble.to_csv(f"{submission_dir}/submission_ensemble.csv", index=False)
print(f"Сохранено: {submission_dir}/submission_ensemble.csv")


print("\nВсе submission-файлы созданы!")


--- Итоги кросс‑валидации ---
lgb: 0.8074
xgb: 0.8133
rf: 0.8012
stack: 0.8060
Генерируем предсказания для lgb...
Сохранено: ./submissions/submission_lgb.csv
Генерируем предсказания для xgb...
Сохранено: ./submissions/submission_xgb.csv
Генерируем предсказания для rf...
Сохранено: ./submissions/submission_rf.csv
Генерируем предсказания для stack...
Сохранено: ./submissions/submission_stack.csv

--- Генерируем ансамблевый submission ---
Сохранено: ./submissions/submission_ensemble.csv

Все submission-файлы созданы!


### Почему именно эти предикторы и модели?
Предикторы (признаки):

Числовые (num_cols): возраст, суммарные траты, логарифмы трат — важны для предсказания, так как отражают поведение пассажиров.

Категориальные (cat_cols): планета отправления, пункт назначения, палуба и т. д. — влияют на вероятность целевого события.

Бинарные (bool_cols): признаки пропусков, наличие трат, взаимодействие признаков — помогают модели лучше понять структуру данных.

Модели:

LightGBM и XGBoost — лидеры в задачах классификации табличных данных, хорошо улавливают нелинейные зависимости.

Random Forest — более устойчивая к переобучению альтернатива бустингу, используется для диверсификации ансамбля.

Stacking — позволяет объединить сильные стороны разных моделей, часто даёт прирост качества.

Отличия моделей:

Бустинг (LightGBM, XGBoost) строит деревья последовательно, корректируя ошибки предыдущих. Хорошо работает с небольшими датасетами, чувствителен к выбросам.

Random Forest строит деревья параллельно, усредняет предсказания. Менее чувствителен к переобучению, но может уступать в точности бустингу.

Stacking использует предсказания базовых моделей как признаки для финальной модели, что позволяет комбинировать их сильные стороны.

Этот подход обеспечивает баланс между качеством, устойчивостью и разнообразием моделей, что важно для успешного участия в соревнованиях по машинному обучению.